## Search AME2016 database
## Import AME database as pandas dataframe

In [ ]:
import numpy as np
import pandas as pd
import fortranformat as ff
import matplotlib as plt
from IPython.display import display
%matplotlib inline

## Create AME16 mass dataframe in appropriate formatting
The input file 'mass16.txt' was downloaded from http://amdc.in2p3.fr/web/masseval.html and is supplemental material of the following publication: Wang, Meng, et al. "The AME2016 atomic mass evaluation (II). Tables, graphs and references." Chinese Physics C 41.3 (2017): 030003..

In [ ]:
df = pd.read_csv('mass16.txt',sep="\t",skiprows=38) # import data

arr_AME = []
line = ff.FortranRecordReader('a1,A3,A5,A5,A5,1x,A3,A4,1x,A13,A11,A11,A9,1x,A2,A11,A9,1x,A3,1x,A12,A11')
for i in range(len(df)):
    row = line.read(str(df.iloc[i].values[0])) 
    #print(i, [row])
    arr_AME += [row]
    #nucl = record._make(line.read(line))
arr_AME = np.array(arr_AME) 

df_AME = pd.DataFrame(arr_AME)
df_AME.columns = ['0','N-Z','N','Z','A','Element','O','ME [keV]','Error ME [keV]','BINDING ENERGY/A [keV]','Error BINDING ENERGY/A [keV]','Decay type','BETA-DECAY ENERGY [keV]','Error BETA-DECAY ENERGY [keV]','Mass_int','ATOMIC MASS [µu]','Error ATOMIC MASS [µu]']
# remove unnecessary columns
df_AME.drop(['0'],axis=1,inplace=True) 
df_AME.drop('Decay type', axis=1, inplace=True)
df_AME.drop('BETA-DECAY ENERGY [keV]', axis=1, inplace=True)
df_AME.drop('Error BETA-DECAY ENERGY [keV]', axis=1, inplace=True)
mask = df_AME['ME [keV]'].str.contains('#') # series containing 'True' if mass excess value has a '#'
df_AME['Extrapolated?'] = mask

# Format column data
df_AME.A = df_AME.A.astype(int)
df_AME['N-Z'] = df_AME['N-Z'].astype(int)
df_AME.N = df_AME.N.astype(int)
df_AME.Z = df_AME.Z.astype(int)
df_AME['Element'] = df_AME['Element'].str.strip()

# Remove '#'s from mass data and change data types to float
df_AME['ME [keV]'] = df_AME['ME [keV]'].str.replace('#', '').astype(float)
df_AME['Error ME [keV]'] = df_AME['Error ME [keV]'].str.replace('#', '').astype(float)
df_AME['BINDING ENERGY/A [keV]'] = df_AME['BINDING ENERGY/A [keV]'].str.replace('#', '').astype(float)
df_AME['Error BINDING ENERGY/A [keV]'] = df_AME['Error BINDING ENERGY/A [keV]'].str.replace('#', '').astype(float)
df_AME['ATOMIC MASS [µu]'] = df_AME['ATOMIC MASS [µu]'].str.replace('#', '').astype(np.float64) 
df_AME['Mass_int'] = df_AME['Mass_int'].astype(int) 
df_AME['ATOMIC MASS [µu]'] = df_AME['ATOMIC MASS [µu]'] + df_AME['Mass_int']*1e06   # calculate full mass value
df_AME.drop(['Mass_int'],axis=1,inplace=True) 
df_AME['Error ATOMIC MASS [µu]'] = df_AME['Error ATOMIC MASS [µu]'].str.replace('#', '').astype(float)

df_AME.set_index(['A'],inplace=True) # set mass number as index

# Output dataframe to .csv (output files require minimal post-processing in EXCEL, see instructions below)
#df_AME.to_csv('AME2016-mass.csv',float_format='%.5f')

df_AME

## CSV post-processing
Import the 'AME2016-mass.csv' file outputted by this notebook into EXCEL using the 'import CSV' function. For the import choose "Unicode (UTF-8) as data origin and deactivate automatic data type recognition. Check the resulting table for correct formatting. Remove the dynamic column handles in the first row (e.g. by simply copying only the values of all columns below to a new worksheet). 
Mark all cells and rightalign their content (optional, for better readability only). 
Delete empty worksheets. Then save this file as a CSV UTF-8 and append '-formatted' to the original filename. 
The CSV UTF8 file will be imported by emgfit.  